In [6]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#!pip install seaborn
import seaborn as sns
from datetime import *
import datetime as dt
from matplotlib.ticker import MaxNLocator
#!pip install regex
import regex
#!pip install emoji
import emoji
from seaborn import *
#!pip install heatmap
#from heatmap import heatmap
#!pip install wordcloud
#from wordcloud import WordCloud , STOPWORDS , ImageColorGenerator
from nltk import *
#from plotly import express as px

C:\Users\Hassan Raza\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
C:\Users\Hassan Raza\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
De

### Python code to extract Date from chat file 

In [20]:
def startsWithDateAndTime(s):
    print('Start with Date and Time function')
    # 11/08/2021, 7:21 pm -
    # pattern = '^([0-9]+)(/)([0-9]+)(/)([0-9][0-9]), ([0-9]+):([0-9][0-9]) (AM|PM) -'
    pattern = '^([0-9]+)(/)([0-9]+)(/)([0-9][0-9]), ([0-9]+):([0-9][0-9]) (am|pm) -'
    result = re.match(pattern, s)
    print(result)
    if result:
        return True
    return False

### Regex pattern to extract username of Author.

In [8]:

def FindAuthor(s):
    patterns = [
        '([w]+):',                        # First Name
        '([w]+[s]+[w]+):',              # First Name + Last Name
        '([w]+[s]+[w]+[s]+[w]+):',    # First Name + Middle Name + Last Name
        '([+]d{2} d{5} d{5}):',         # Mobile Number (India no.)
        '([+]d{2} d{3} d{3} d{4}):',   # Mobile Number (US no.)
        '([w]+)[u263a-U0001f999]+:',    # Name and Emoji              
    ]
    pattern = '^' + '|'.join(patterns)
    result = re.match(pattern, s)
    if result:
        return True
    return False

### Extracting Date, Time, Author and message from the chat file.

In [9]:

def getDataPoint(line):   
    splitLine = line.split(' - ') 
    dateTime = splitLine[0]
    date, time = dateTime.split(', ') 
    message = ' '.join(splitLine[1:])
    if FindAuthor(message): 
        splitMessage = message.split(': ') 
        author = splitMessage[0] 
        message = ' '.join(splitMessage[1:])
    else:
        author = None
    return date, time, author, message


In [21]:
### Finally creating a dataframe and storing all data inside that dataframe.
parsedData = [] # List to keep track of data so it can be used by a Pandas dataframe
### Uploading exported chat file
conversationPath = 'GroupChat.txt' # chat file
with open(conversationPath, encoding="utf-8") as fp:
    ### Skipping first line of the file because contains information related to something about end-to-end encryption
    fp.readline()
    fp.readline()
    fp.readline()
    messageBuffer = [] 
    date, time, author = None, None, None
    while True:
        line = fp.readline()
        print(line)
        if not line: 
            break
        line = line.strip() 
        if startsWithDateAndTime(line): 
            print(100)
            if len(messageBuffer) > 0:
                print('!!!!!!!!!!!!!!!!!')
                parsedData.append([date, time, author, ' '.join(messageBuffer)]) 
            messageBuffer.clear() 
            date, time, author, message = getDataPoint(line) 
            messageBuffer.append(message) 
        else:
            print(200)
            messageBuffer.append(line)
    
print(len(parsedData))

df = pd.DataFrame(parsedData, columns=['Date', 'Time', 'Author', 'Message']) # Initialising a pandas Dataframe.
### changing datatype of "Date" column.
df["Date"] = pd.to_datetime(df["Date"])

11/08/2021, 7:21 pm - HR: Why you created this Group😄

Start with Date and Time function
None
200
11/08/2021, 7:21 pm - Saad MS_34: 😂

Start with Date and Time function
None
200
11/08/2021, 7:22 pm - HR: Rockstar ko b add kr ly

Start with Date and Time function
None
200
11/08/2021, 7:22 pm - HR: 😏

Start with Date and Time function
None
200
11/08/2021, 7:22 pm - Saad MS_34: 😂😂😂

Start with Date and Time function
None
200
11/08/2021, 7:23 pm - HR: Otherwise me jaa rha classmate

Start with Date and Time function
None
200
11/08/2021, 7:23 pm - HR: I want Rockstar here

Start with Date and Time function
None
200
11/08/2021, 7:23 pm - HR: 🙌🏼

Start with Date and Time function
None
200
11/08/2021, 7:23 pm - Saad MS_34: 😂😂😂

Start with Date and Time function
None
200
11/08/2021, 7:24 pm - Saad MS_34: Me krda gal

Start with Date and Time function
None
200
11/08/2021, 7:24 pm - HR: Oo

Start with Date and Time function
None
200
11/08/2021, 7:24 pm - HR: Ok

Start with Date and Time function


In [11]:
df

,Date,Time,Author,Message


In [4]:
conversationPath = 'GroupChat.txt' # chat file
with open(conversationPath, encoding="utf-8") as fp:
    ### Skipping first line of the file because contains information related to something about end-to-end encryption
    #fp.readlines()
    data = (fp.readlines())

print(len(data))

1240
